In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.tradingview.com/markets/stocks-usa/market-movers-high-dividend/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', class_='table-Ngq2xrcG')
rows = table.find_all('tr', attrs={'data-rowkey': True})

data_rowkeys = [row['data-rowkey'] for row in rows]
print(data_rowkeys)

['NASDAQ:RILY', 'NASDAQ:IEP', 'NASDAQ:GIPR', 'NYSE:MNR', 'NYSE:TPVG', 'NASDAQ:OFS', 'NASDAQ:TCPC', 'NASDAQ:DALN', 'NASDAQ:RWAY', 'NASDAQ:ICON', 'NASDAQ:PTMN', 'NYSE:VOC', 'NASDAQ:XP', 'NASDAQ:TRIN', 'NASDAQ:OXSQ', 'NYSE:KEN', 'NASDAQ:GECC', 'NASDAQ:PSEC', 'NASDAQ:NHTC', 'NYSE:CATO', 'NYSE:NEP', 'NASDAQ:OCSL', 'NYSE:GSBD', 'NASDAQ:WHF', 'NASDAQ:SQFT', 'NYSE:FSK', 'NYSE:SAR', 'NYSE:CMTG', 'NASDAQ:FAT', 'NYSE:DSX', 'NASDAQ:MRCC', 'NYSE:CION', 'NASDAQ:MNDO', 'NYSE:TXO', 'NASDAQ:USEA', 'AMEX:COHN', 'NYSE:SCM', 'NASDAQ:WBA', 'NYSE:KRP', 'NASDAQ:DMLP', 'NYSE:NAT', 'NASDAQ:ARLP', 'NASDAQ:SLRC', 'NASDAQ:NMFC', 'NYSE:GHI', 'NYSE:AMBP', 'NASDAQ:HTCR', 'NYSE:PSBD', 'NYSE:BBDC', 'NYSE:PFLT', 'NYSE:SGHC', 'NYSE:BXSL', 'NASDAQ:GBDC', 'NYSE:UAN', 'NYSE:KSS', 'NYSE:BCSF', 'NYSE:OBDC', 'NASDAQ:TTEC', 'NYSE:CAPL', 'NYSE:DKL', 'NYSE:BSM', 'NASDAQ:BGFV', 'NYSE:EVC', 'NYSE:OBDE', 'NASDAQ:XRX', 'NASDAQ:SBLK', 'NASDAQ:MARPS', 'NASDAQ:CGBD', 'NYSE:SFL', 'NYSE:BWMX', 'NYSE:USAC', 'NASDAQ:ARCC', 'NASDAQ:CCAP', '

In [2]:
symbols = [item.split(':')[1] for item in data_rowkeys]
print(symbols)


['RILY', 'IEP', 'GIPR', 'MNR', 'TPVG', 'OFS', 'TCPC', 'DALN', 'RWAY', 'ICON', 'PTMN', 'VOC', 'XP', 'TRIN', 'OXSQ', 'KEN', 'GECC', 'PSEC', 'NHTC', 'CATO', 'NEP', 'OCSL', 'GSBD', 'WHF', 'SQFT', 'FSK', 'SAR', 'CMTG', 'FAT', 'DSX', 'MRCC', 'CION', 'MNDO', 'TXO', 'USEA', 'COHN', 'SCM', 'WBA', 'KRP', 'DMLP', 'NAT', 'ARLP', 'SLRC', 'NMFC', 'GHI', 'AMBP', 'HTCR', 'PSBD', 'BBDC', 'PFLT', 'SGHC', 'BXSL', 'GBDC', 'UAN', 'KSS', 'BCSF', 'OBDC', 'TTEC', 'CAPL', 'DKL', 'BSM', 'BGFV', 'EVC', 'OBDE', 'XRX', 'SBLK', 'MARPS', 'CGBD', 'SFL', 'BWMX', 'USAC', 'ARCC', 'CCAP', 'BGS', 'CSWC', 'TSLX', 'WES', 'TFSL', 'BRY', 'FDUS', 'OMF', 'CBRL', 'HIHO', 'EPM', 'SPOK', 'WLKP', 'LIEN', 'PAX', 'AB', 'SHIP', 'GLAD', 'DHT', 'MO', 'AY', 'CVI', 'DSWL', 'VTS', 'HTGC', 'WU', 'ET']


In [11]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Initialize an empty dictionary to store DataFrames for each stock
stock_data = {}

# Get data for each symbol
for symbol in symbols:
    try:
        # Fetch the stock data
        stock = yf.Ticker(symbol)
        
        # Get dividend history
        dividends = stock.dividends

        # Create a DataFrame for this stock
        df = pd.DataFrame(dividends)
        df.columns = ['Dividend']
        df.index.name = 'Date'

        # Add columns for stock price 3 days before and after dividend
        df['Price_Before'] = None
        df['Price_After'] = None

        for date in df.index:
            # Get stock price 3 days before dividend
            before_date = date - timedelta(days=3)
            history_before = stock.history(start=before_date, end=date)
            if not history_before.empty:
                df.loc[date, 'Price_Before'] = history_before['Close'].iloc[-1]

            # Get stock price 3 days after dividend
            after_date = date + timedelta(days=3)
            history_after = stock.history(start=date, end=after_date)
            if not history_after.empty:
                df.loc[date, 'Price_After'] = history_after['Close'].iloc[-1]

        # Add this DataFrame to the dictionary
        stock_data[symbol] = df

        print(f"Processed {symbol}")
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")

# Display the resulting dictionary of DataFrames
for symbol, df in stock_data.items():
    print(f"\nData for {symbol}:")
    print(df)


Processed RILY


$IEP: possibly delisted; no price data found  (1d 1989-03-24 00:00:00-05:00 -> 1989-03-27 00:00:00-05:00)
$IEP: possibly delisted; no price data found  (1d 1993-12-24 00:00:00-05:00 -> 1993-12-27 00:00:00-05:00)
$IEP: possibly delisted; no price data found  (1d 2013-02-16 00:00:00-05:00 -> 2013-02-19 00:00:00-05:00)


Processed IEP
Processed GIPR
Processed MNR
Processed TPVG
Processed OFS
Processed TCPC
Processed DALN
Processed RWAY
Processed ICON


$PTMN: possibly delisted; no price data found  (1d 2008-07-04 00:00:00-04:00 -> 2008-07-07 00:00:00-04:00)
$PTMN: possibly delisted; no price data found  (1d 2010-04-02 00:00:00-04:00 -> 2010-04-05 00:00:00-04:00)
$PTMN: possibly delisted; no price data found  (1d 2016-01-01 00:00:00-05:00 -> 2016-01-04 00:00:00-05:00)
$PTMN: possibly delisted; no price data found  (1d 2016-07-02 00:00:00-04:00 -> 2016-07-05 00:00:00-04:00)


Processed PTMN
Processed VOC
Processed XP
Processed TRIN


...

Processed MO
Processed AY
Processed CVI


$DSWL: possibly delisted; no price data found  (1d 2008-07-04 00:00:00-04:00 -> 2008-07-07 00:00:00-04:00)
$DSWL: possibly delisted; no price data found  (1d 2015-01-17 00:00:00-05:00 -> 2015-01-20 00:00:00-05:00)
$DSWL: possibly delisted; no price data found  (1d 2016-07-02 00:00:00-04:00 -> 2016-07-05 00:00:00-04:00)


Processed DSWL
Processed VTS
Processed HTGC
Processed WU


$ET: possibly delisted; no price data found  (1d 2006-12-31 00:00:00-05:00 -> 2007-01-03 00:00:00-05:00)


Processed ET

Data for RILY:
                           Dividend Price_Before Price_After
Date                                                        
2014-11-14 00:00:00-05:00  0.029703     4.541427    4.147872
2015-05-20 00:00:00-04:00  0.059406     5.696206    5.623311
2015-11-20 00:00:00-05:00  0.059406     4.946431    5.103626
2016-08-18 00:00:00-04:00  0.029703      4.93595    5.004287
2016-11-25 00:00:00-05:00  0.250000     8.962508    9.188203
2017-03-02 00:00:00-05:00  0.260000     8.406802    8.138268
2017-05-19 00:00:00-04:00  0.160000     7.699842    7.649951
2017-08-17 00:00:00-04:00  0.130000      8.75864    8.579779
2017-11-21 00:00:00-05:00  0.120000    10.088922   10.522189
2018-03-19 00:00:00-04:00  0.160000     10.91607   11.148685
2018-05-18 00:00:00-04:00  0.120000    11.829526   11.812405
2018-08-15 00:00:00-04:00  0.300000    12.982233   12.403958
2018-11-15 00:00:00-05:00  0.160000    10.119781   10.341552
2019-03-18 00:00:00-04:00  0.080000     9.991387    9.92

In [16]:
# Initialize a dictionary to store the capital history for each symbol
capital_histories = {}

# Define start and end dates
start_date = pd.to_datetime('2018-01-01', utc=True)
end_date = pd.to_datetime('2024-10-01', utc=True)

# Iterate through each symbol and its corresponding DataFrame
for symbol, df in stock_data.items():
    capital = 10000  # Starting capital for each symbol
    capital_history = []  # List to store capital values over time
    
    # Filter DataFrame for the specified date range
    df_filtered = df.loc[start_date:end_date]
    
    # Iterate through each dividend date
    for date, row in df_filtered.iterrows():
        if pd.isna(row['Price_Before']) or pd.isna(row['Price_After']) or pd.isna(row['Dividend']):
            continue  # Skip if we don't have price data or dividend data
        
        # Calculate number of shares we can buy
        shares = capital // row['Price_Before']
        
        # Calculate new capital after buying shares
        capital_after_buy = capital - (shares * row['Price_Before'])
        
        # Calculate capital after dividend payout
        capital_after_dividend = capital_after_buy + (shares * row['Dividend'])
        
        # Calculate final capital after selling shares
        capital = capital_after_dividend + (shares * row['Price_After'])
        
        # Store the capital value for this date
        capital_history.append((date, capital))
    
    # Store the capital history for this symbol
    capital_histories[symbol] = capital_history

# Create a DataFrame for each symbol's capital history
for symbol, history in capital_histories.items():
    df = pd.DataFrame(history, columns=['Date', 'Capital'])
    df.set_index('Date', inplace=True)
    capital_histories[symbol] = df

# Print the capital history for each symbol
for symbol, df in capital_histories.items():
    print(f"\nCapital history for {symbol}:")
    print(df)

# Calculate and print final capitals
final_capitals = {}
for symbol, df in capital_histories.items():
    if not df.empty:
        final_capitals[symbol] = df['Capital'].iloc[-1]
    else:
        print(f"Warning: No capital history data for {symbol}")
        final_capitals[symbol] = 10000  # Use initial capital if no data

sorted_capitals = sorted(final_capitals.items(), key=lambda x: x[1], reverse=True)

print("\nFinal capital for each symbol, ranked:")
for symbol, capital in sorted_capitals:
    print(f"{symbol}: ${capital:.2f}")


Capital history for RILY:
                                Capital
Date                                   
2018-03-19 00:00:00-04:00  10359.635771
2018-05-18 00:00:00-04:00  10449.654621
2018-08-15 00:00:00-04:00  10225.921740
2018-11-15 00:00:00-05:00  10611.509730
2019-03-18 00:00:00-04:00  10621.744176
2019-05-14 00:00:00-04:00  11217.615286
2019-08-14 00:00:00-04:00  11826.962003
2019-11-13 00:00:00-05:00  12263.759439
2020-03-16 00:00:00-04:00   9759.382713
2020-05-29 00:00:00-04:00   9998.377934
2020-08-13 00:00:00-04:00  10155.001005
2020-11-09 00:00:00-05:00  10832.521441
2021-03-09 00:00:00-05:00  12567.813444
2021-05-14 00:00:00-04:00  13117.876440
2021-08-12 00:00:00-04:00  13430.025248
2021-11-08 00:00:00-05:00  13687.532267
2022-03-08 00:00:00-05:00  14434.644884
2022-05-10 00:00:00-04:00  14533.856852
2022-08-10 00:00:00-04:00  15781.034876
2022-11-14 00:00:00-05:00  15476.199793
2023-03-09 00:00:00-05:00  13086.266588
2023-05-15 00:00:00-04:00  14694.026297
2023-08-10 00

In [28]:
stock_data

{'RILY':                            Dividend Price_Before Price_After
 Date                                                        
 2014-11-14 00:00:00-05:00  0.029703     4.541427    4.147872
 2015-05-20 00:00:00-04:00  0.059406     5.696206    5.623311
 2015-11-20 00:00:00-05:00  0.059406     4.946431    5.103626
 2016-08-18 00:00:00-04:00  0.029703      4.93595    5.004287
 2016-11-25 00:00:00-05:00  0.250000     8.962508    9.188203
 2017-03-02 00:00:00-05:00  0.260000     8.406802    8.138268
 2017-05-19 00:00:00-04:00  0.160000     7.699842    7.649951
 2017-08-17 00:00:00-04:00  0.130000      8.75864    8.579779
 2017-11-21 00:00:00-05:00  0.120000    10.088922   10.522189
 2018-03-19 00:00:00-04:00  0.160000     10.91607   11.148685
 2018-05-18 00:00:00-04:00  0.120000    11.829526   11.812405
 2018-08-15 00:00:00-04:00  0.300000    12.982233   12.403958
 2018-11-15 00:00:00-05:00  0.160000    10.119781   10.341552
 2019-03-18 00:00:00-04:00  0.080000     9.991387    9.921024


In [30]:

# Define your start and end dates
start_date = pd.Timestamp('2014-01-01')
end_date = pd.Timestamp('2024-10-01')

# Create a date range and initialize the centralized DataFrame
date_range = pd.date_range(start=start_date, end=end_date)
central_df = pd.DataFrame(index=date_range)

# Process each stock's DataFrame and merge it into the centralized DataFrame
for symbol, df in stock_data.items():
    
    # Convert the index to datetime without timezone and remove time
    df.index = pd.to_datetime(df.index).normalize()

    # Select the 'Dividend' column
    dividend_series = df['Dividend']
    
    # Reindex to match the central DataFrame's index
    dividend_series = dividend_series.reindex(central_df.index)
    
    # Add the dividend series to the central DataFrame
    central_df[symbol] = dividend_series

# Display the centralized DataFrame
print(central_df)


DatetimeIndex(['2014-11-14 00:00:00-05:00', '2015-05-20 00:00:00-04:00',
               '2015-11-20 00:00:00-05:00', '2016-08-18 00:00:00-04:00',
               '2016-11-25 00:00:00-05:00', '2017-03-02 00:00:00-05:00',
               '2017-05-19 00:00:00-04:00', '2017-08-17 00:00:00-04:00',
               '2017-11-21 00:00:00-05:00', '2018-03-19 00:00:00-04:00',
               '2018-05-18 00:00:00-04:00', '2018-08-15 00:00:00-04:00',
               '2018-11-15 00:00:00-05:00', '2019-03-18 00:00:00-04:00',
               '2019-05-14 00:00:00-04:00', '2019-08-14 00:00:00-04:00',
               '2019-11-13 00:00:00-05:00', '2020-03-16 00:00:00-04:00',
               '2020-05-29 00:00:00-04:00', '2020-08-13 00:00:00-04:00',
               '2020-11-09 00:00:00-05:00', '2021-03-09 00:00:00-05:00',
               '2021-05-14 00:00:00-04:00', '2021-08-12 00:00:00-04:00',
               '2021-11-08 00:00:00-05:00', '2022-03-08 00:00:00-05:00',
               '2022-05-10 00:00:00-04:00', '2022-0